In [15]:
import os
os.environ["JAVA_HOME"] = "/Users/debasis/devtools/jdk-11.0.10.jdk/Contents/Home/"
print(os.getenv("JAVA_HOME"))

/Users/debasis/devtools/jdk-11.0.10.jdk/Contents/Home/


In [16]:
import pyterrier as pt
from transformers import AutoTokenizer
from collections import Counter
from pyterrier.measures import *
import pandas as pd

In [17]:
def printTopFrequentTerms(index_ref):
    # Access the lexicon
    lexicon = index_ref.getLexicon()
    
    # Extract terms and their frequencies
    term_freqs = []
    iterator = lexicon.iterator()
    while iterator.hasNext():
        entry = iterator.next()
        term_freqs.append((entry.getKey(), entry.getValue().getFrequency()))
    
    # Sort by frequency and get the top 50 terms
    top_terms = sorted(term_freqs, key=lambda x: x[1], reverse=True)[:50]
    
    # Print results
    for term, freq in top_terms:
        print(f"{term}: {freq}")

In [26]:
dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019')

WORD_INDEX_DIR="./wordindex"
word_index_ref = pt.IndexFactory.of(WORD_INDEX_DIR)

#print(word_index_ref.getCollectionStatistics().toString())

tfidf = pt.terrier.Retriever(word_index_ref, wmodel="TF_IDF", num_results=100)
#tfidf_gpt = pt.terrier.Retriever(index_ref, wmodel="TF_IDF", properties={"termpipelines" : ""}, num_results=100)  # No tokenization

rm3 = tfidf >> pt.rewrite.RM3(word_index_ref) >> tfidf
results = pt.Experiment([tfidf, rm3], dataset.get_topics(), dataset.get_qrels(), eval_metrics=[nDCG@10, AP(rel=2), AP(rel=3)],names=["tfidf", "rm3"])
print (results[["nDCG@10", "AP(rel=2)", "AP(rel=3)"]])

13:21:26.306 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
    nDCG@10  AP(rel=2)  AP(rel=3)
0  0.478310   0.232189   0.163141
1  0.525136   0.258130   0.194894


In [24]:
dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019')

# Load Queries from File
query_file = "trecdl/pass_2019.gpt-tok.queries"  # Change to your actual query file
queries = pd.read_csv(query_file, sep='\t', names=["qid", "query"], dtype={"qid": str, "query": str})

# Load the Indexed Collection
index_dir = "./gpt_index"  # Path to your indexed data
index_ref = pt.IndexFactory.of(index_dir)

#print(index_ref.getCollectionStatistics().toString())

tfidf_gpt = pt.terrier.Retriever(index_ref, wmodel="TF_IDF", num_results=100)  # No tokenization
rm3_gpttok = tfidf_gpt >> pt.rewrite.RM3(index_ref, fb_terms=10, fb_docs=10) >> tfidf_gpt

results = pt.Experiment([tfidf_gpt, rm3_gpttok],
              queries, dataset.get_qrels(), eval_metrics=[nDCG@10, AP(rel=2), AP(rel=3)],
              names=["tfidf-gpt-tokens", "tfidf-gpt-tokens-rm3"])
print (results[["nDCG@10", "AP(rel=2)", "AP(rel=3)"]])


13:18:10.825 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 2 GiB of memory would be required.
    nDCG@10  AP(rel=2)  AP(rel=3)
0  0.421618   0.187597   0.148047
1  0.473803   0.210617   0.165043
